# Transfer Learning with tensorflow Part 1: Feature Extraction

Transfer learning is leveraging a working model's architecture and learned patterns for our own problem.
1. Can leverage an existing neural network architecture proven to work on problems similar to our own. 
2. Can leverage a working neural network architecture which has already learned patterns on similar data to our own,then we can adapt those patterns to our own data

## Downloading and becoming one with the data

In [3]:
# Get data(10% of 10 food classes from Food101)
import zipfile

# Unzip the downloaded file
zip_ref=zipfile.ZipFile("10_food_classes_10_percent.zip","r")
zip_ref.extractall()
zip_ref.close()

In [4]:
# How many images in each folder?
import os

# Walk through 10 percent data directory and list number of files 
for dirpath,dirnames,filesnames in os.walk("10_food_classes_10_percent"):
    print(f"There are {len(dirnames)} directories and {len(filesnames)} images in '{dirpath}'.")

There are 2 directories and 0 images in '10_food_classes_10_percent'.
There are 10 directories and 0 images in '10_food_classes_10_percent\test'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\chicken_curry'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\chicken_wings'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\fried_rice'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\grilled_salmon'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\hamburger'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\ice_cream'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\pizza'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\ramen'.
There are 0 directories and 250 images in '10_food_classes_10_percent\test\steak'.
There are 0 directories and 250 images in '10_food_classes_10_percent

## Creating data loaders(preparing the data)

We'll use the `ImageDataGenerator` class to load in our images in batches

In [5]:
# Setup data inputs
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMAGE_SHAPE=(224,224)
BATCH_SIZE=32

train_dir="10_food_classes_10_percent/train/"
test_dir="10_food_classes_10_percent/test/"

train_datagen=ImageDataGenerator(rescale=1/225.)
test_datagen=ImageDataGenerator(rescale=1/225.)

print("Training Images")
train_data_10_percent=train_datagen.flow_from_directory(train_dir,
                                                        target_size=IMAGE_SHAPE,
                                                        batch_size=BATCH_SIZE,
                                                        class_mode="categorical")
print("Testing images")
test_data_10_percent=test_datagen.flow_from_directory(test_dir,
                                                      target_size=IMAGE_SHAPE,
                                                      batch_size=BATCH_SIZE,
                                                      class_mode="categorical")

Training Images
Found 750 images belonging to 10 classes.
Testing images
Found 2500 images belonging to 10 classes.
